<img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100"/> 
<h1> unSkript Runbooks </h1>
<div class="alert alert-block alert-success">
    <b> This runbook demonstrates How to copy AMI from one region to another AWS region using unSkript legos.</b>
</div>

<br>

<center><h2>Copy AMI to All Given AWS Regions</h2></center>

# Steps Overview
    Used a AWS CLI command to copy AMI from one region to another AWS region.
    "aws ec2 copy-image --source-image-id <ami-id> --source-region <source-region> --region <dest-region> --name <ami-name>"

Here we will use unSkript Run Command via AWS CLI Lego. This lego takes aws_command, AMI_Id, SourceRegion, DestinationRegion, Name as input. These inputs are used to copy AMI from one region to another AWS region.

In [25]:
#
# Copyright (c) 2021 unSkript.com
# All rights reserved.
#
from pydantic import BaseModel, Field
import pprint
from beartype import beartype


@beartype
def aws_execute_cli_command_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_execute_cli_command(handle, aws_command: str, AMI_Id: str, SourceRegion: str, DestinationRegion: str, Name: str) -> str:
    aws_command = aws_command.format(AMI_Id=AMI_Id, SourceRegion=SourceRegion, DestinationRegion=DestinationRegion, Name=Name)
    result = handle.aws_cli_command(aws_command)
    if result is None or result.returncode != 0:
        print(
            f"Error while executing command ({aws_command}): {result}")
        return str()
    
    return result.stdout


task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "AMI_Id": "AMI_Id",
    "DestinationRegion": "iter_item",
    "SourceRegion": "SourceRegion",
    "aws_command": "\\"aws ec2 copy-image --source-image-id {AMI_Id} --source-region {SourceRegion} --region {DestinationRegion} --name {Name}\\"",
    "Name": "Name"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "DestinationRegions",
    "iter_parameter": "DestinationRegion"
    }''')
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_execute_cli_command, lego_printer=aws_execute_cli_command_printer, hdl=hdl, args=args)

   In this Runbook, we demonstrated the use of unSkript's AWS actions to perform copy AMI from one region to another AWS region.
   To view the full platform capabilities of unSkript please visit https://us.app.unskript.io